# 1. Neural Network Training

In [9]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler

from Ranker.neuralnetwork import NeuralNetwork
from DataHandling.train_data import load_data


### 1.1. Load training and test data

In [10]:
feature_indices = [108, 18, 103, 3, 8, 13, 125, 126, 107, 17, 102, 2, 7, 12, 105, 15, 100, 0, 5, 10]

In [11]:
# load training data
X_train, y_train = load_data(path="../../../data/MSLR-WEB10K/Fold1/train.txt", nrows=100000, feature_indices=feature_indices)
#X_test, y_test = load_data(path="../../../data/MSLR-WEB10K/Fold1/test.txt", nrows=10000)



In [12]:
y_trans = torch.zeros(len(y_train), 5)
for i, label in enumerate(y_train):
    y_trans[i, 0:int(label)+1] = 1 


### 1.2. Create model

In [13]:
model = NeuralNetwork(n_features=len(feature_indices), n_hidden=10)

In [14]:
### define optimizer and loss fct
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
loss = nn.CrossEntropyLoss()

### 1.3. Train Model

In [15]:
for epoch in range(10):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss = model.train_loop(torch.Tensor(X_train), y_trans, loss, optimizer)

Epoch 1
-------------------------------
loss: 122.954323  [   32/100000]
loss: 1.091780  [16032/100000]
loss: 1.190276  [32032/100000]
loss: 1.351556  [48032/100000]
loss: 1.443593  [64032/100000]
loss: 1.529235  [80032/100000]
loss: 1.795762  [96032/100000]
Epoch 2
-------------------------------
loss: 1.557791  [   32/100000]
loss: 2.067106  [16032/100000]
loss: 1.110698  [32032/100000]
loss: 1.979844  [48032/100000]
loss: 1.688128  [64032/100000]
loss: 1.656128  [80032/100000]
loss: 2.215463  [96032/100000]
Epoch 3
-------------------------------
loss: 1.305716  [   32/100000]
loss: 1.347183  [16032/100000]
loss: 1.680132  [32032/100000]
loss: 1.775565  [48032/100000]
loss: 1.461952  [64032/100000]
loss: 1.499614  [80032/100000]
loss: 1.457599  [96032/100000]
Epoch 4
-------------------------------
loss: 1.389264  [   32/100000]
loss: 1.474380  [16032/100000]
loss: 1.935525  [32032/100000]
loss: 1.421622  [48032/100000]
loss: 1.557544  [64032/100000]
loss: 1.805623  [80032/100000]
l

### 1.4. Save Model

In [16]:
# save mdoel
model.save("../models/nn.pth")

model saved in  ../models/nn.pth
